In [1]:
%load_ext autoreload

In [ ]:
%autoreload 2

from dotenv import load_dotenv
import sys
import os
from pathlib import Path
load_dotenv()
# Add the parent directory of 'src' to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.config_loader import config
from src.utils import anthropic_generate, extract_json_from_llm_response, save_json
from src.chat import get_challenge_generation_prompt, create_html_challenges, get_html_challenge_inputs
COLLECTION = "WarmUp150"

In [4]:
from src.gcs_storage import get_stories_from_collection, get_story_dialogue_path, get_story_challenges_path, upload_to_gcs, read_from_gcs, get_story_translated_challenges_path

all_stories = get_stories_from_collection(collection=COLLECTION)

## Generate scenarios

In [ ]:
for story_name in all_stories:
    story_dialogue = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=get_story_dialogue_path(story_name=story_name, collection=COLLECTION))
    prompt = get_challenge_generation_prompt(story_dialogue)
    scenario_dicts = extract_json_from_llm_response(anthropic_generate(prompt, max_tokens=5000))

    upload_to_gcs(obj=scenario_dicts, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=get_story_challenges_path(story_name=story_name, collection=COLLECTION))

## If scenarios already exist start here

In [ ]:
scenario_dicts = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=get_story_challenges_path(story_name="story_midnight_garden_mystery", collection="LM1000"))
challenges = get_html_challenge_inputs(scenario_dicts) # this is the first time config.TARGET_LANGUAGE_NAME is used
chat_webpage_file = create_html_challenges(challenges, story_name="story_midnight_garden_mystery")

In [ ]:
upload_to_gcs(file_path=chat_webpage_file, bucket_prefix=f"{config.TARGET_LANGUAGE_NAME}/{STORY_NAME}")